# Initialize: remove border cells + tracking all movies in a folder

Notebook to automatically do initial steps of EpiCure:
* **removing the border cells** (optionnaly, cells that are partly outside of the image)
* **performs the tracking** (with the default parameters)
* **save** the results

These 3 steps will be done on all the movies in a given folder (in all sub-folders), provided that there is an associated segmentation file with the same name as the input image + given extension string (for example `_imagename_epyseg.tif`).

*This notebook is part of EpiCure release, see https://gitlab.pasteur.fr/gletort/epicure for more informations*

In [1]:
#### Parent directory to process. It will go through all the sub-folders
main_path = "../data/small/"
#### Name of the associated segmentation file for each image. It must have the same name as the image, with the additional segmentation_extension
segmentation_extension = "_epyseg"
#### Option clear border of EpiCure: ON (True) or OFF (False)
remove_border_cells = True

In [ ]:
import epicure.epicuring as epicure
import os

def doOneMovie( imgname ):
    """ Process one movie: clear (or not) the border cells and performs tracking """
    print( "EpiCuring movie "+imgname+".tif" )
    epic = epicure.EpiCure()
    epic.verbose = 0         ## Choose the level of printing infos during process (0: minimal to 3: debug informations)
    epic.load_movie( imgname+".tif" )
    epic.go_epicure( "epics", imgname+segmentation_extension+".tif" )  ## start EpiCure, load the segmentation, prepare everything
    if remove_border_cells:
        epic.editing.border_size.setText("1") ## Choose the border size parameter to remove the cells that are within the given distance of the image border
        epic.editing.remove_border()  ## EpiCure option to remove all cells that are touching the image border
    epic.tracking.do_tracking()       ## Performs tracking with the default parameters. If you have saved preferences, it will use it.
    epic.save_epicures()              ## save the results in the ouput "epics" folder(s)
    
    
### Main loop, go through all subdirectories and process movies for which the associated segmentation file correspond to the segmentation_extension name
for root, dirs, files in os.walk( main_path ):
    for file in files:
        if file.endswith( segmentation_extension+".tif" ):
            imgname = file[:len(file)-4-len(segmentation_extension)]
            imgname = os.path.join( root, imgname ) 
            doOneMovie( imgname )